Automatische FEA für eine Geometrieanalyse:

Ziel: Filterung der Materialien

Annahmen:   
- volles Volumen, 
- schwächstes Modul in alle Richtungen annehmen für Isotropes Material

- Druck, oder Strecktest simulation in alle sechs Raumrichtungen 

- Von Mises Yield criterion für plastische Einschätzung


Prozessablauf: 
1. Der Kunde erstellt seinen Auftrag und wir bekommen 
            

In [36]:
import numpy as np
from skfem import *
import pytetwild
import meshio
from skfem.models.elasticity import (linear_elasticity, lame_parameters)
from skfem.helpers import dot, sym_grad
from skfem.visuals.matplotlib import draw, plot
import  matplotlib.pyplot as plt

# Material properties for PLA and ABS
materials = {
    "PLA": {
        "Youngs Modulus": 2636e6,  # Pa (2636 MPa)
        "Poisson Ratio": 0.327,
        "Yielding Stress": 54.4e6,  # Pa (54.4 MPa)
    },
    "ABS": {
        "Youngs Modulus": 2400e6,  # Pa (2400 MPa)
        "Poisson Ratio": 0.37,
        "Yielding Stress": 26.84e6  # Pa (26.84 MPa)
    }
}

# Different mechanical tests
tests = {
    'compression test': {
        'displacements': {
            'left': ('right', 'u^1', 'app_press'),
            'right': ('left', 'u^1', '-app_press'),
            'up': ('down', 'u^2', '-app_press'),
            'down': ('up', 'u^2', 'app_press'),
            'front': ('back', 'u^3', '-app_press'),
            'back': ('front', 'u^3', 'app_press')
        }
    },
    'tensile test': {
        'displacements': {
            'left': ('right', 'u^1', '-app_press'),
            'right': ('left', 'u^1', 'app_press'),
            'up': ('down', 'u^2', 'app_press'),
            'down': ('up', 'u^2', '-app_press'),
            'front': ('back', 'u^3', 'app_press'),
            'back': ('front', 'u^3', '-app_press')
        }
    }
}

def Analyse(material, pressure, test, object):
    """
    Perform finite element analysis on the given object.

    Args:
    material (str): Material to use for analysis ('PLA' or 'ABS')
    pressure (float): Applied pressure on the object
    test (str): Choose 'tensile test' or 'compression test'
    object (str): Path to the STL file of the object
    """
    # Read the STL file and create the tetrahedral mesh
    stl = meshio.read(f"C:/Users/AlexP/Desktop/BA/STL_Files/{object}")
    vertices, tetrahedras = pytetwild.tetrahedralize(stl.points, stl.cells_dict["triangle"])
    cells = meshio.CellBlock('tetra', tetrahedras)
    p = np.array([vertices[:, 0], vertices[:, 1], vertices[:, 2]], dtype=np.float64)
    t = np.array(tetrahedras, dtype=np.float64).T
    mesh = MeshTet(p, t)
    
    # Define FE mesh elements
    e1 = ElementTetP1()
    e = ElementVector(e1)
    ib = Basis(mesh, e, MappingIsoparametric(mesh, e1), 3)

    # Extract material properties
    young_modulus = materials[material]["Youngs Modulus"]
    poisson_ratio = materials[material]["Poisson Ratio"]

    # Assemble stiffness matrix
    K = asm(linear_elasticity(*lame_parameters(young_modulus, poisson_ratio)), ib)

    # Define boundary conditions based on the test type (tensile or compression)
    dofs = {
        'left': ib.get_dofs(lambda x: np.isclose(x[0], x[0].min())),
        'right': ib.get_dofs(lambda x: np.isclose(x[0], x[0].max())),
        'up': ib.get_dofs(lambda x: np.isclose(x[1], x[1].max())),
        'down': ib.get_dofs(lambda x: np.isclose(x[1], x[1].min())),
        'front': ib.get_dofs(lambda x: np.isclose(x[2], x[2].max())),
        'back': ib.get_dofs(lambda x: np.isclose(x[2], x[2].min()))
    }

    # Set displacements based on the selected test
    displacements = tests[test]['displacements']
    
    u = ib.zeros()  # Initialize the displacement field
    u[ib.get_dofs('right').nodal['u^1']] = pressure
    
    u = solve(*condense(K,x=u, D=ib.get_dofs({'left','right'})))
    sf = 1.0
    m = m.translated(sf * u[basis.nodal_dofs])
    draw(m)
    return  # Return displacement and stress for further analysis

# Example usage
pressure = 2e8  # Applied pressure in Pascals (1 MPa)
material = 'PLA'  # Choose between 'PLA' or 'ABS'
test_type = 'compression test'  # Choose between 'tensile test' or 'compression test'
object_file = "300_polygon_sphere_100mm.STL"  # Path to the STL file

result = Analyse(material, pressure, test_type, object_file)




Starting tetrahedralization process...


Transforming over 1000 elements to C_CONTIGUOUS.


Tetrahedralization complete.
Number of vertices: 1278
Number of tetrahedra: 5717
Prepared numpy array for points.
Prepared numpy array for tetrahedra.
Tetrahedralization process completed successfully.


ValueError: Boundary 'right' not found.

In [23]:
import numpy as np
from skfem import *
import pytetwild
import meshio
from skfem.models.elasticity import (linear_elasticity, lame_parameters, linear_stress)
from skfem.helpers import dot, sym_grad
from skfem.visuals.matplotlib import draw, plot
import  matplotlib.pyplot as plt



materials = {
    "PLA": {
        "Youngs Modulus": 2636e6,  # Pa (2636 MPa)
        "Poisson Ratio": 0.327,
        "Yielding Stress": 54.4e6,  # Pa (54.4 MPa)
    },
    "ABS": {
        "Youngs Modulus": 2400e6,  # Pa (2400 MPa)
        "Poisson Ratio": 0.37,
        "Yielding Stress": 26.84e6  # Pa (26.84 MPa)
    }
}
def visualize(m_shifted, vonmises, dgbh):
    shiftedbase = Basis(m_shifted,dgbh)
    ax = plot(shiftedbase,vonmises, shading='gouraud',colorbar =r"$\sigma_{\mathrm{mises}}$")
    draw(m_shifted, ax = ax)
    return ax
def yield_stress(array, x):
    e = []
    p = []
    for i in range(len(array)):
        if array[i] > x:
            p.append(i)
        
    return {'plastic': p}

def FEM_PressTest(material, pressure, object):
       # Read the STL file and create the tetrahedral mesh
    stl = meshio.read(f"C:/Users/AlexP/Desktop/BA/STL_Files/{object}")
    vertices, tetrahedras = pytetwild.tetrahedralize(stl.points, stl.cells_dict["triangle"])
    cells = meshio.CellBlock('tetra', tetrahedras)
    p = np.array([vertices[:, 0], vertices[:, 1], vertices[:, 2]], dtype=np.float64)
    t = np.array(tetrahedras, dtype=np.float64).T
    mesh = MeshTet(p, t)
       # Define FE mesh elements
    e1 = ElementTetP1()
    e = ElementVector(e1)
    ib = Basis(mesh, e, MappingIsoparametric(mesh, e1), 3)
    
    young_modulus = materials[material]["Youngs Modulus"]
    poisson_ratio = materials[material]["Poisson Ratio"]
    lame_params = lame_parameters(young_modulus, poisson_ratio)
    
    K = asm(linear_elasticity(*lame_params), ib)
    
        # Define boundary conditions based on the test type (tensile or compression)
    dofs = {
        'left': ib.get_dofs(lambda x: np.isclose(x[0], x[0].min())),
        'right': ib.get_dofs(lambda x: np.isclose(x[0], x[0].max())),
        'up': ib.get_dofs(lambda x: np.isclose(x[1], x[1].max())),
        'down': ib.get_dofs(lambda x: np.isclose(x[1], x[1].min())),
        'front': ib.get_dofs(lambda x: np.isclose(x[2], x[2].max())),
        'back': ib.get_dofs(lambda x: np.isclose(x[2], x[2].min()))
    }
    
    displacements = {
        'left': ('right', 'u^1', pressure),
        'right': ('left', 'u^1', -pressure),
        'up': ('down', 'u^2', -pressure),
        'down': ('up', 'u^2', pressure),
        'front': ('back', 'u^3', -pressure),
        'back': ('front', 'u^3', pressure)
    }
    
    
    
    
    results = {}
    for direction, (opposite, component, press_value) in displacements.items():
        u = ib.zeros()

        # Initialize force array
        F = np.zeros(u.shape)

        # Correctly index the nodal DOFs
        direction_dofs = dofs[direction].nodal[component].astype(int).flatten()
        F[direction_dofs] = press_value

        # Fix boundary conditions
        fixed_dofs = np.hstack([dofs[opposite].all()])
        u[fixed_dofs] = 0

        # Set DOFs and solve the system
        free_dofs = np.setdiff1d(np.arange(K.shape[0]), fixed_dofs)
        K_free = K[free_dofs][:, free_dofs]
        F_free = F[free_dofs]
        u_free = solve(K_free, F_free)
        u[free_dofs] = u_free
        # compute the strain tensor
         # Calculate final shift
        final_shift = u[direction_dofs]

        # Implement shifts
        u[direction_dofs] = final_shift
        I = ib.complement_dofs(np.concatenate([dofs[direction], dofs[opposite]]))
        u = solve(*condense(K, x=u, I=I))
        
        dgbh = ib.with_element(ElementTetP0())
        
        sf = 1000
        
        
        # Translate and save the mesh
        m_shifted = mesh.translated(sf * u[ib.nodal_dofs])
        # draw(m_shifted) 
        m_shifted.save(f"C:/Users/AlexP/Desktop/BA/Meshes/Solutions/TESTS_{object}_{direction}.vtk", {"affected": u[ib.nodal_dofs][0]})
        
        #vonmises calculation 
        s = {}
        dgb = ib.with_element(ElementTetP0())
        
        
        up = ib.interpolate(u)

        C = linear_stress(*lame_params)
        
        
        for i in [0, 1]:
            for j in [0, 1]:
                s[i, j] = dgb.project(C(sym_grad(up))[i, j])

        s[2, 2] = poisson_ratio * (s[0, 0] + s[1, 1])

        vonmises = np.sqrt(.5 * ((s[0, 0] - s[1, 1]) ** 2 +
                                (s[1, 1] - s[2, 2]) ** 2 +
                                (s[2, 2] - s[0, 0]) ** 2 +
                                6. * s[0, 1] ** 2))
        material_yield_stress = materials[material]['Yielding Stress']
        
        
        
        yield_results = yield_stress(vonmises, material_yield_stress)
        results[direction] = yield_results
    return results
        
    



def FEM_TensileTest(material, pressure, object):
       # Read the STL file and create the tetrahedral mesh
    stl = meshio.read(f"C:/Users/AlexP/Desktop/BA/STL_Files/{object}")
    vertices, tetrahedras = pytetwild.tetrahedralize(stl.points, stl.cells_dict["triangle"])
    cells = meshio.CellBlock('tetra', tetrahedras)
    p = np.array([vertices[:, 0], vertices[:, 1], vertices[:, 2]], dtype=np.float64)
    t = np.array(tetrahedras, dtype=np.float64).T
    mesh = MeshTet(p, t)
       # Define FE mesh elements
    e1 = ElementTetP1()
    e = ElementVector(e1)
    ib = Basis(mesh, e, MappingIsoparametric(mesh, e1), 3)
    
    young_modulus = materials[material]["Youngs Modulus"]
    poisson_ratio = materials[material]["Poisson Ratio"]
    lame_params = lame_parameters(young_modulus, poisson_ratio)
    
    K = asm(linear_elasticity(*lame_params), ib)
    
        # Define boundary conditions based on the test type (tensile or compression)
    dofs = {
        'left': ib.get_dofs(lambda x: np.isclose(x[0], x[0].min())),
        'right': ib.get_dofs(lambda x: np.isclose(x[0], x[0].max())),
        'up': ib.get_dofs(lambda x: np.isclose(x[1], x[1].max())),
        'down': ib.get_dofs(lambda x: np.isclose(x[1], x[1].min())),
        'front': ib.get_dofs(lambda x: np.isclose(x[2], x[2].max())),
        'back': ib.get_dofs(lambda x: np.isclose(x[2], x[2].min()))
    }
    
    displacements = {
        'left': ('right', 'u^1', -pressure),
        'right': ('left', 'u^1', pressure),
        'up': ('down', 'u^2', pressure),
        'down': ('up', 'u^2', -pressure),
        'front': ('back', 'u^3', pressure),
        'back': ('front', 'u^3', -pressure)
    }
    
    
    
    
    results = {}
    for direction, (opposite, component, press_value) in displacements.items():
        u = ib.zeros()

        # Initialize force array
        F = np.zeros(u.shape)

        # Correctly index the nodal DOFs
        direction_dofs = dofs[direction].nodal[component].astype(int).flatten()
        F[direction_dofs] = press_value

        # Fix boundary conditions
        fixed_dofs = np.hstack([dofs[opposite].all()])
        u[fixed_dofs] = 0

        # Set DOFs and solve the system
        free_dofs = np.setdiff1d(np.arange(K.shape[0]), fixed_dofs)
        K_free = K[free_dofs][:, free_dofs]
        F_free = F[free_dofs]
        u_free = solve(K_free, F_free)
        u[free_dofs] = u_free
        # compute the strain tensor
         # Calculate final shift
        final_shift = u[direction_dofs]

        # Implement shifts
        u[direction_dofs] = final_shift
        I = ib.complement_dofs(np.concatenate([dofs[direction], dofs[opposite]]))
        u = solve(*condense(K, x=u, I=I))
        
        dgbh = ib.with_element(ElementTetP0())
        
        sf = 1000
        
        
        # Translate and save the mesh
        m_shifted = mesh.translated(sf * u[ib.nodal_dofs])
        # draw(m_shifted) 
        m_shifted.save(f"C:/Users/AlexP/Desktop/BA/Meshes/Solutions/TensileTESTS_{object}_{direction}.vtk", {"affected": u[ib.nodal_dofs][0]})
        m_shifted = MeshTet()
        #vonmises calculation 
        s = {}
        dgb = ib.with_element(ElementTetP1())
        
        
        up = ib.interpolate(u)

        C = linear_stress(*lame_params)
        
        
        for i in [0, 1]:
            for j in [0, 1]:
                s[i, j] = dgb.project(C(sym_grad(up))[i, j])

        s[2, 2] = poisson_ratio * (s[0, 0] + s[1, 1])

        vonmises = np.sqrt(.5 * ((s[0, 0] - s[1, 1]) ** 2 +
                                (s[1, 1] - s[2, 2]) ** 2 +
                                (s[2, 2] - s[0, 0]) ** 2 +
                                6. * s[0, 1] ** 2))
        material_yield_stress = materials[material]['Yielding Stress']
        
        
        visualize(m_shifted, vonmises, dgb)
        yield_results = yield_stress(vonmises, material_yield_stress)
        
        print(len(vonmises))
        results[direction] = yield_results
        
    return results

result = FEM_TensileTest("PLA", 1e7, "Cube_3d_printing_sample (1).stl")    
print(result)


Transforming over 1000 elements to C_CONTIGUOUS.


Starting tetrahedralization process...
Tetrahedralization complete.
Number of vertices: 2169
Number of tetrahedra: 9661
Prepared numpy array for points.
Prepared numpy array for tetrahedra.
Tetrahedralization process completed successfully.


AttributeError: 'CellBasis' object has no attribute 'refdom'

In [21]:
print(len(vonmises))

NameError: name 'vonmises' is not defined